In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from math import log
import pprint

In [7]:
def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    labels = ['age', 'whether_job', 'whether_house', 'credit', 'category']
    # 返回数据集和每个维度的名称
    return datasets, labels

In [8]:
datasets , labels = create_data()
train_data = pd.DataFrame(datasets,columns = labels)
print(train_data)
value_count = train_data['category'].value_counts()
print(value_count)

   age whether_job whether_house credit category
0   青年           否             否     一般        否
1   青年           否             否      好        否
2   青年           是             否      好        是
3   青年           是             是     一般        是
4   青年           否             否     一般        否
5   中年           否             否     一般        否
6   中年           否             否      好        否
7   中年           是             是      好        是
8   中年           否             是    非常好        是
9   中年           否             是    非常好        是
10  老年           否             是    非常好        是
11  老年           否             是      好        是
12  老年           是             否      好        是
13  老年           是             否    非常好        是
14  老年           否             否     一般        否
category
是    9
否    6
Name: count, dtype: int64


In [9]:
#计算熵
def calc_entropy(train_data):
    
    value_count = train_data['category'].value_counts()
    data_sum = value_count.sum()
    entropy = - sum((i/data_sum)*log(i/data_sum,2) if i != 0 else 0 for i in value_count)
    return entropy
calc_entropy(train_data)


0.9709505944546686

In [10]:
#计算条件熵
def calc_condition_entropy(train_data):
    condition_entropy = {}
    for i, feature in enumerate(train_data.columns):
        if feature == 'category':
            continue
        feature_count = []
        df_categories = train_data[feature].unique()
        value_count = train_data[feature].value_counts()
        for category in df_categories:
            df = train_data
            category_rows = df[df[feature] == category]
            category_num = len(category_rows)
            a = category_num/len(train_data)*calc_entropy(category_rows)
            feature_count.append(a)
        feature_entropy = sum(feature_count)
        condition_entropy[feature] = feature_entropy
    return condition_entropy
        
        
        
            
            
            
    
            
        
        
        

In [11]:
calc_condition_entropy(train_data)

{'age': 0.8879430945988998,
 'whether_job': 0.6473003963031123,
 'whether_house': 0.5509775004326937,
 'credit': 0.6079610319175832}

In [12]:
# 信息增益
def info_gain(train_data):
    info_gain = {}
    entropy = calc_entropy(train_data)
    condition_entropy = calc_condition_entropy(train_data)
    for key,value in condition_entropy.items():
        
        feature_info_gain = entropy - value
        info_gain[key] = feature_info_gain
    key_with_min_value = max(info_gain, key=info_gain.get)
    min_value = max(info_gain.values())
 
        
    return info_gain , '特征({})的信息增益最大，选择为根节点特征'.format(key_with_min_value)

In [13]:
info_gain(train_data)

({'age': 0.08300749985576883,
  'whether_job': 0.32365019815155627,
  'whether_house': 0.4199730940219749,
  'credit': 0.36298956253708536},
 '特征(whether_house)的信息增益最大，选择为根节点特征')

In [14]:
# 定义节点类 二叉树
class Node:
    def __init__(self, root=True, label=None, feature_name=None):
        self.root = root
        self.label = label
        self.feature_name = feature_name
        self.feature = feature
        self.tree = {}
        self.result = {
            'label:': self.label,
            'feature': self.feature,
            'tree': self.tree
        }

    def __repr__(self):
        return '{}'.format(self.result)

    def add_node(self, val, node):
        self.tree[val] = node

    def predict(self, features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature]].predict(features)

In [38]:
#
class D_Tree():
    def __init__(self,epsilon =  0.01):
        self.epsilon = epsilon
        self._tree = {}
    @staticmethod
    #计算熵
    def calc_entropy(train_data):
        
    
        value_count = train_data['category'].value_counts()
        data_sum = value_count.sum()
        entropy = - sum((i/data_sum)*log(i/data_sum,2) if i != 0 else 0 for i in value_count)
        return entropy
     
    #计算条件熵
    def calc_condition_entropy(train_data):
        condition_entropy = {}
        for i, feature in enumerate(train_data.columns):
            if feature == 'category':
                continue
            feature_count = []
            df_categories = train_data[feature].unique()
            value_count = train_data[feature].value_counts()
            for category in df_categories:
                df = train_data
                category_rows = df[df[feature] == category]
                category_num = len(category_rows)
                a = category_num/len(train_data)*calc_entropy(category_rows)
                feature_count.append(a)
                feature_entropy = sum(feature_count)
                condition_entropy[feature] = feature_entropy
        return condition_entropy
    # 信息增益
    def info_gain(self,train_data):
        info_gain_ = {}
        entropy = calc_entropy(train_data)
        condition_entropy = calc_condition_entropy(train_data)
        for key,value in condition_entropy.items():
        
            feature_info_gain = entropy - value
            info_gain_[key] = feature_info_gain
        best_feature = max(info_gain_, key=info_gain_.get)
        best_feature_info_gain = max(info_gain_.values())
 
        
        return best_feature,best_feature_info_gain
    def train(self, train_data):
        """
        input:数据集D(DataFrame格式)，特征集A，阈值eta
        output:决策树T
        """
        x_train, y_train, features = train_data.iloc[:, :
                                               -1], train_data.iloc[:,
                                                                    -1], train_data.columns[:
                                                                                            -1]
        # 1,若D中实例属于同一类Ck，则T为单节点树，并将类Ck作为结点的类标记，返回T
        if len(y_train.value_counts()) == 1:
            return Node(root=True, label=y_train.iloc[0])

        # 2, 若A为空，则T为单节点树，将D中实例树最大的类Ck作为该节点的类标记，返回T
        if len(features) == 0:
            return Node(
                root=True,
                label=y_train.value_counts().sort_values(
                    ascending=False).index[0])

        # 3,计算最大信息增益 同5.1,Ag为信息增益最大的特征
        best_feature, best_feature_info_gain = self.info_gain(train_data)
        

        # 4,Ag的信息增益小于阈值eta,则置T为单节点树，并将D中是实例数最大的类Ck作为该节点的类标记，返回T
        if best_feature_info_gain < self.epsilon:
            return Node(
                root=True,
                label=y_train.value_counts().sort_values(
                    ascending=False).index[0])

        # 5,构建Ag子集
        node_tree = Node(
            root=False, feature_name=best_feature)

        feature_list = train_data[best_feature].value_counts().index
        for f in feature_list:
            sub_train_df = train_data.loc[train_data[best_feature] ==
                                          f].drop([best_feature], axis=1)

            # 6, 递归生成树
            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f, sub_tree)

        # pprint.pprint(node_tree.tree)
        return node_tree
    def fit(self, train_data):
        self._tree = self.train(train_data)
        return self._tree
    def predict(self, X_test):
        return self._tree.predict(X_test)

In [39]:

dt = D_Tree()
tree = dt.fit(train_data)

tree



{'label:': None, 'feature': None, 'tree': {'否': {'label:': None, 'feature': None, 'tree': {'否': {'label:': '否', 'feature': None, 'tree': {}}, '是': {'label:': '是', 'feature': None, 'tree': {}}}}, '是': {'label:': '是', 'feature': None, 'tree': {}}}}

In [116]:
import numpy as np


class LeastSqRTree:
    def __init__(self, train_X, y, epsilon):
        # 训练集特征值
        self.x = train_X
        # 类别
        self.y = y
        # 特征总数
        self.feature_count = train_X.shape[1]
        # 损失阈值
        self.epsilon = epsilon
        # 回归树
        self.tree = None

    def _fit(self, x, y, feature_count, epsilon):
        # 选择最优切分点变量j与切分点s
        (j, s, minval, c1, c2) = self._divide(x, y, feature_count)
        # 初始化树
        tree = {"feature": j, "value": x[s, j], "left": None, "right": None}
        if minval.any() < self.epsilon or len(y[np.where(x[:, j] <= x[s, j])]) <= 1:
            tree["left"] = c1
        else:
            tree["left"] = self._fit(x[np.where(x[:, j] <= x[s, j])],
                                     y[np.where(x[:, j] <= x[s, j])],
                                     self.feature_count, self.epsilon)
        if minval.any() < self.epsilon or len(y[np.where(x[:, j] > s)]) <= 1:
            tree["right"] = c2
        else:
            tree["right"] = self._fit(x[np.where(x[:, j] > x[s, j])],
                                      y[np.where(x[:, j] > x[s, j])],
                                      self.feature_count, self.epsilon)
        return tree

    def fit(self):
        self.tree = self._fit(self.x, self.y, self.feature_count, self.epsilon)

    @staticmethod
    def _divide(x, y, feature_count):
        # 初始化损失误差
        cost = np.zeros((feature_count, len(x)))
        # 公式5.21
        for i in range(feature_count):
            for k in range(len(x)):
                # k行i列的特征值
                value = x[k, i]
                y1 = y[np.where(x[:, i] <= value)]
                c1 = np.mean(y1)
                y2 = y[np.where(x[:, i] > value)]
                c2 = np.mean(y2)
                y1[:] = y1[:] - c1
                y2[:] = y2[:] - c2
                cost[i, k] = np.sum(y1 * y1) + np.sum(y2 * y2)
        # 选取最优损失误差点
        cost_index = np.where(cost == np.min(cost))
        # 选取第几个特征值
        j = cost_index[0][0]
        # 选取特征值的切分点
        s = cost_index[1][0]
        # 求两个区域的均值c1,c2
        c1 = np.mean(y[np.where(x[:, j] <= x[s, j])])
        c2 = np.mean(y[np.where(x[:, j] > x[s, j])])
        return j, s, cost[cost_index], c1, c2

train_X = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],[1,2,4,6,7,8,33,45,5,5]]).T
y = np.array([4.50, 4.75, 4.91, 5.34, 5.80, 7.05, 7.90, 8.23, 8.70, 9.00])

model_tree = LeastSqRTree(train_X, y, .2)
model_tree.fit()
model_tree.tree



{'feature': 0,
 'value': 5,
 'left': {'feature': 0,
  'value': 3,
  'left': {'feature': 0,
   'value': 1,
   'left': 4.5,
   'right': {'feature': 0, 'value': 2, 'left': 4.75, 'right': 4.91}},
  'right': {'feature': 0, 'value': 4, 'left': 5.34, 'right': 5.8}},
 'right': {'feature': 0,
  'value': 7,
  'left': {'feature': 0, 'value': 6, 'left': 7.05, 'right': 7.9},
  'right': {'feature': 0,
   'value': 8,
   'left': 8.23,
   'right': {'feature': 0, 'value': 9, 'left': 8.7, 'right': 9.0}}}}